# Trabajo final

En cada celda de código se explica mediante comentarios qué hacer y cómo. 

Respetad los huecos entre comentarios para rellenar con código lo que se pide justo debajo de cada comentario. 

1. Consulta y filtrado de valores nulos (1 puntos)
2. Consultas con Query (2.5 puntos)
3. Consultas indexación booleana (1 punto)
3. Discretización y Variables dummies (1.5 puntos)
4. Union de datasets, agrupamiento y más consultas con grupos (3 puntos)
6. Consultas libres (1 puntos)

## 1 Carga modificación y filtrado de nulos

In [5]:
import pandas as pd

# importar medallas_jjoo.csv
data_medallas = pd.read_csv('medallas_jjoo.csv')

# muestra el tamaño del dataset
print(data_medallas.size)

# mostrar las primeras filas de medallas para comprender su estructura y comprobar la importación
data_medallas.head()

10752


,Year,Host_country,Host_city,Country_Name,Country_Code,Gold,Silver,Bronze
0,1896,Greece,Athens,Great Britain,GBR,2,3,2
1,1896,Greece,Athens,Hungary,HUN,2,1,3
2,1896,Greece,Athens,France,FRA,5,4,2
3,1896,Greece,Athens,United States,USA,11,7,2
4,1896,Greece,Athens,Germany,GER,6,5,2


In [7]:
# comprobar los si existen valores nulos, cuántos son y en qué columnas
print( data_medallas.isnull().sum())

Year             0
Host_country     0
Host_city        0
Country_Name     0
Country_Code    86
Gold             0
Silver           0
Bronze           0
dtype: int64


In [ ]:
# Vamos a eliminar la columna country_code que además es la que tiene nulos
data_medallas.drop(columns='country_code')

# Las columnas son renombradas a españo: año, host_pais, host_ciudad, pais, oro, plata bronce
# host se refiere a la ciudad o pais organizador de los juegos



# mostramos las primeras filas para comprobar que la eliminación y renombrado de columnas ha sido correcto


In [4]:
# Comprueba si existen filas duplicadas


## 2. Consultas query 
Realiza las consultas de cada comentario utilizando query. 

Algunas necesitarán alguna acción más, como filtrar una columna y alguna operación, ordenar valores, etc. 

 

In [5]:
# Consulta 1: Mostrar el número y tipo de medallas España en los JJOO de Barcelona 1992


In [6]:
# Consulta 2: Mostrar cuántas medallas de oro ha ganado españa en toda la historia


In [7]:
# Consulta 3: Mostrar el medallero de Atenas 2004 de forma descendente en columna medallas de oro.


In [8]:
# Consulta 4: ¿Cuántas veces se han celebrado los JJOO en Francia? Mostrar solamente el número con un mensaje.


In [9]:
# Consulta 5: Mostrar las filas de los paises 'Spain', 'France', 'Germany', 'Italy',  que han ganado más de 10 medallas de oro


## 3. Indexación booleana

Replica las consultas 3 y 5 del apartado 2, pero usando indexación booleana en lugar de usar query.

In [ ]:
# Consulta 3 


In [ ]:
# Consulta 5 


## 4. Discretización y variable dummy 


In [10]:
# Copia el dataframe medallas en medallas2 para trabajar con el


In [11]:
# Genera una nueva columna que sea "puntuacion" asignando:
    # 3 puntos a cada medalla de oro
    # 2 puntos a cada medalla de plata
    # 1 punto a cada medalla de bronce





# muestra las primeras filas para comprobar que la nueva columna ha sido creada correctamente



In [12]:
# Discretización: Crea una nueva columna "puntuacion_categoria" que divida la puntuación en 3 categorías: baja, media y alta
# Puedes utilizar qcut para generar intervalos de forma equitativa




# muestra las primeras filas para comprobar que la nueva columna ha sido creada correctamente


In [13]:
# Crea un dataframe "dummydf" con variables dummies para la columna puntuacion_categoria con el prefijo "puntos"




# añade las variables dummy al dataframe medallas2 con join 



# Muestra las primeras filas para comprobar que las variables dummies han sido creadas correctamente


## 5. Union de datasets y consultas con agrupamiento

+ Cada consulta es 0.6 puntos. 
+ La unión de ambos datasets es 1.2 puntos.

In [14]:
# Consulta 1: Usando agrupamiento, mostrar el número total de medallas de oro, plata y bronce de cada país



In [15]:
# Importa el dataset paises.csv en el dataframe paises
# Este csv contiene información sobre la población y el PIB per capita de paises, 
    # además del continente y subregion al que pertenecen. 



# muestra las primeras filas para comprender su estructura y comprobar la importación


In [16]:
# Compruba que no haya nulos es el dataframe paises. 


In [17]:
# Consulta 2: Con agrupamiento, a partir de dataset paises, consultar la media del PIB y poblacición de cada región



In [18]:
# Une los dataframes medallas y paises en un nuevo dataframe llamado medallas_paises usando la columna pais
# Debes hacer un merge interno.



# muestra las primeras filas para comprobar que la unión ha sido correcta


In [19]:
# Comprueba que no existan nulos en el dataframe medallas_paises. Si los hay, quizás tienes el ejercicio mal.


In [20]:
# Consulta 3: Mostrar el número total de medallas de oro, plata y bronce de cada región, 
    # pero solamente de las regiones con más de 300 medallas de oro
    # La salida debe ser ordenada de forma descentende según el número de medallas de oro


## 6. Consultas libres

Realiza 2 consultas adicionales y diferentes a las anteriores, usando query o indexación booleana, que tengan además el uso de  agrupamiento y funciones de agregación y/o creación de intervalos. 

Las consultas se deben hacer sobre el dataset resultante de la unión de ambos csv realizada en el punto 4.

La nota dependerá de la complejidad de la consulta.

In [ ]:
# Consulta 1


In [ ]:
# Consulta 2
